## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
from sklearn import metrics
from sklearn.metrics import f1_score, recall_score, accuracy_score
import seaborn as sns
from sklearn.metrics import confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

## Load Data

In [3]:
df = pd.read_csv('/Users/alexlei/Downloads/accepted_2007_to_2018Q4.csv', index_col=0, low_memory=False)

## Drop Columns

In [4]:
# Drop unnecessary columns
df = df.drop(df.loc[:, 'hardship_flag':'settlement_term'].columns, axis=1)
df = df.drop(['Total amount funded in policy code 1: 1465324575', 'Total amount funded in policy code 2: 521953170'])

In [ ]:
df = df.loc[df['loan_status'].notnull()]

In [ ]:
df['fico_range_high'].fillna(df['fico_range_high'].mean(), inplace=True)

In [ ]:
df['fico_range_low'].fillna(df['fico_range_low'].mean(), inplace=True)

In [ ]:
# Fill Loan Status Null values

In [ ]:
df['loan_status'].value_counts()

## Assigning X & y

In [5]:
mapping_loan_status = {
    'Fully Paid': 1,
    'Does not meet the credit policy. Status:Fully Paid': 1,
    'Current': 0.75,
    'Late (16-30 days)': 0.5,
    'Late (31-120 days)': 0.25,
    'In Grace Period': 0.25,
    'Charged Off': 0,
    'Default': 0,
    'Does not meet the credit policy. Status:Charged Off': 0
}

# Convert loan_status to numerical values based on the mapping
df['loan_status_numeric'] = df['loan_status'].map(mapping_loan_status)

In [6]:
target_list = [0 if i in ['Default', 'Charged Off'] else 1 for i in df['loan_status']]

df['TARGET'] = target_list
df['TARGET'].value_counts()

1    1992100
0     268599
Name: TARGET, dtype: int64

In [7]:
df['TARGET'].isnull().any()

False

In [8]:
temp = df[['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 
    'installment', 'annual_inc', 'delinq_2yrs', 'fico_range_low', 
    'fico_range_high', 'inq_last_6mths', 'total_pymnt', 'total_rec_prncp',
   'tot_cur_bal', 'mort_acc']]

In [9]:
X = temp
y = df['TARGET']

### Imputers

In [10]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [11]:
imputer = imp_mean.fit(X)
X = imputer.transform(X)

In [12]:
# Assign weights based on the mapping
sample_weights = np.where(df['loan_status_numeric'] == 1, 1, df['loan_status_numeric'])

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, sample_weights_train, sample_weights_test = train_test_split(X, y, sample_weights, 
                                                                                               test_size=0.2, random_state=42, 
                                                                                               stratify=df['TARGET'], shuffle=True)

## Implementing a Scaler

In [13]:
# Transform X - Min-Max Scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Creating a classification report for the model

In [14]:
from sklearn.metrics import classification_report
#print(classification_report(y_test,y_pred))

In [15]:
def print_score(true, pred, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

## Creating LogisticRegression Model

In [16]:
# Create and train the model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

#### Prediction

In [17]:
y_pred = model.predict(X_test)

test_acc = accuracy_score(y_test, y_pred)
print('The Accuracy for Test Set is {}'.format(test_acc*100))

The Accuracy for Test Set is 89.94183217587472


## Autoencoder

In [18]:
# Define the # of input features (# of columns in 'df')
num_features = X_train.shape[1]
latent_dim = 10

In [21]:
# Define the encoder architecture
input_data = Input(shape=(num_features,))

encoded = Dense(20, activation='relu')(input_data)
encoded = Dropout(0.3)(encoded)
encoded = Dense(latent_dim, activation='sigmoid')(encoded)

In [22]:
# Latent space representation
latent_representation = Dense(latent_dim, activation='sigmoid')(encoded)

In [23]:
# # Define encoder model
# encoder_model = Model(input_data, encoded)
encoder_model = Model(input_data, latent_representation)

In [24]:
# Decoder layers
decoded = Dense(20, activation='relu')(encoded)

# Output layer
decoded = Dense(num_features, activation='sigmoid')(decoded)

In [27]:
# Define autoencoder model
autoencoder = Model(input_data, decoded)
#autoencoder_model = Model(encoder_model.input, decoder_output)

# Compile autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [28]:
# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=10, batch_size=10, shuffle=True)

Epoch 1/10
180856/180856 [==============================] - 63s 349us/step - loss: 0.0026
Epoch 2/10
180856/180856 [==============================] - 65s 359us/step - loss: 0.0020
Epoch 3/10
180856/180856 [==============================] - 63s 351us/step - loss: 0.0017
Epoch 4/10
180856/180856 [==============================] - 62s 345us/step - loss: 0.0016
Epoch 5/10
180856/180856 [==============================] - 63s 350us/step - loss: 0.0016
Epoch 6/10
180856/180856 [==============================] - 64s 351us/step - loss: 0.0016
Epoch 7/10
180856/180856 [==============================] - 64s 354us/step - loss: 0.0015
Epoch 8/10
180856/180856 [==============================] - 64s 355us/step - loss: 0.0015
Epoch 9/10
180856/180856 [==============================] - 64s 355us/step - loss: 0.0015
Epoch 10/10
180856/180856 [==============================] - 64s 355us/step - loss: 0.0015


In [29]:
# Extract the learned features from the encoder
encoder = Model(input_data, encoded)
encoded_data = encoder.predict(X)

70647/70647 [==============================] - 14s 194us/step


In [30]:
encoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14)]              0         
                                                                 
 dense (Dense)               (None, 20)                300       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
Total params: 510 (1.99 KB)
Trainable params: 510 (1.99 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Calculate reconstruction error for each column
reconstruction_errors = np.mean(np.square(X - autoencoder.predict(X)), axis=0)

70647/70647 [==============================] - 14s 195us/step


In [32]:
# Identify the column with the smallest reconstruction error (potential feature)
best_feature_index = np.argmin(reconstruction_errors)
best_feature_name = df.columns[best_feature_index]

In [33]:
print(f"The best feature to use as input is: {best_feature_name}")

The best feature to use as input is: installment


## Scoring the Model

In [34]:
y_pred = model.predict(X_test)

In [35]:
f1_score = metrics.f1_score(y_test, y_pred, average='weighted')
print(f1_score)

0.876690149787038


In [36]:
accuracy_score = metrics.accuracy_score(y_test, y_pred)
print(accuracy_score)

0.8994183217587473


In [37]:
input_sample = temp.sample(1)
input_sample

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,total_pymnt,total_rec_prncp,tot_cur_bal,mort_acc
id,,,,,,,,,,,,,,
133316879,12750.0,12750.0,12750.0,21.85,351.06,30000.0,0.0,670.0,674.0,0.0,1724.35,616.57,14233.0,3.0


In [38]:
prediction = model.predict(input_sample.values)
prediction

array([1])

### No LabelEncoder on df['loan_status'], no Dropout in neither encoder and decoder
### Still need to work on pulling specific row from sample prediction + work on Deployment

### Below PCA shows NA in all component columns including the 'loan_status' column

## PCA
### n_components = 4

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=4)

principalComponents = pca.fit_transform(X)

principalDf = pd.DataFrame(data = principalComponents,
              columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])

In [ ]:
finalDf = principalDf.merge(df[['TARGET']], left_on='principal component 1', right_on='TARGET')

In [ ]:
finalDf

In [ ]:
principalDf.isnull().any()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Make an instance of the Model
pca = PCA(.95)

In [ ]:
pca.fit(X_train)

In [ ]:
train_X = pca.transform(X_train)
test_X = pca.transform(X_test)

In [ ]:
Xfinal = principalDf
yfinal = df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(Xfinal,yfinal,test_size=0.3)

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(X=X_train, y=y_train)

In [ ]:
logistic.predict(X_test)

In [ ]:
score = logistic.score(X_test,y_test)

In [ ]:
score

### Using Random Forest Classifier

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
rf_clf.score(X_test, y_test)

### Using Logistic Regression model

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

## Pipeline

In [39]:
# Define PCA pipeline for training data
pca_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values using mean imputation
    ('scaler', MinMaxScaler()),  # Scale features to a range (default is [0, 1])
    ('pca', PCA(n_components=4))  # Apply PCA for dimensionality reduction
])

# Fit PCA pipeline to training data
pca_pipeline.fit(X_train)

# Transform testing data using PCA
X_test_pca = pca_pipeline.transform(X_test)

# Fit a Linear Regression model to transformed training data
model = LinearRegression()
model.fit(pca_pipeline.transform(X_train), y_train)

# Reverse PCA transformation on predicted values
y_pred = model.predict(X_test_pca)

# Reverse PCA transformation on actual values for calculating mean squared error
X_test_reverse_pca = pca_pipeline.named_steps['pca'].inverse_transform(X_test_pca)

# Calculate mean squared error
mse = mean_squared_error(X_test, X_test_reverse_pca)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.0003167254687457965
